### Create Public Vector Database

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import CohereEmbeddings
import json
import random
from langchain.vectorstores import Chroma

with open('config.json', 'r') as f:
    config = json.load(f)
    game_name = config['game_name']
    character = config['characters'][0]
    character_name = character['name']

# Load the public information file
with open(f'{game_name}/public_info.txt', errors='ignore') as f:
    public_info = f.read()

# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=500,
    length_function=len,
    separators=['\n\n\n', '\n\n', '\n', '.', ',', ' ']
)

# Split the text
public_info_list = text_splitter.split_text(public_info)

# Randomly select an API key from apikeys.json
with open('apikeys.json', 'r') as f:
    api_keys = json.load(f)['api_keys']
selected_key = api_keys[random.randint(0, len(api_keys) - 1)]

# Initialize the Cohere embedding model with user agent fallback
embeddings = CohereEmbeddings(
    cohere_api_key=selected_key, 
    model="large",  # Specify model size
    user_agent="CharacterProfileGenerator/1.0"  # Set user agent
)

# Set the persist directory
persist_directory = f'{game_name}/public_vectordb'

# Create and persist the vector database
vectordb = Chroma.from_texts(public_info_list, embeddings, persist_directory=persist_directory)
vectordb.persist()


FileNotFoundError: [Errno 2] No such file or directory: 'config.json'